## Spaceship Titanic

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
pd.read_csv('sample_submission.csv').head(5) # Formato de subuida.

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
df.sample(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
3016,3272_01,Earth,NaN,G/518/S,TRAPPIST-1e,40.0,False,0.0,0.0,895.0,0.0,0.0,Clarry Kellynn,True
4915,5243_01,Earth,False,F/1005/S,TRAPPIST-1e,24.0,False,1231.0,0.0,0.0,1.0,3.0,Lanedy Mclardson,True
2989,3236_01,Earth,True,G/524/P,55 Cancri e,1.0,False,0.0,0.0,0.0,0.0,0.0,Garyan Woody,True
4865,5190_01,Earth,False,F/994/S,TRAPPIST-1e,13.0,False,0.0,0.0,0.0,0.0,1513.0,Jillie Barrilloney,False
1449,1529_02,Europa,False,A/16/S,TRAPPIST-1e,30.0,False,0.0,0.0,0.0,0.0,0.0,Krab Slable,True
8678,9267_01,Europa,True,E/607/S,TRAPPIST-1e,21.0,False,0.0,0.0,0.0,0.0,0.0,Arraid Opshaft,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3588,3862_02,Mars,False,E/256/S,TRAPPIST-1e,21.0,False,2627.0,0.0,597.0,0.0,0.0,Blakes Ple,True
6312,6673_01,Earth,False,E/425/P,TRAPPIST-1e,45.0,False,1595.0,0.0,0.0,0.0,13.0,Sonnie Phillis,False
1431,1509_01,Mars,False,E/95/P,TRAPPIST-1e,32.0,NaN,1240.0,0.0,58.0,0.0,0.0,Sumpix Dal,False


### Etapa de preprocesamiento de los datos:

* PassengerId: Vemos que tiene la forma *gggg_pp* donde la *g* representa el grupo en el que se encuentra el pasajero y *p* representa el número de gente del grupo. Generalmente cada grupo representa una familia. Dejamos esta parámetro para el final. Es tipo string y tiene 8693 registros.

#### HomePlanet:

Origin planet od the passenger. We've got 201 nulls in the parameter. Must create dummy variables for each planet. How should we treat category parameters in which rows this category is null?

In [9]:
df['HomePlanet'].unique()

array(['Europa', 'Earth', 'Mars', nan], dtype=object)